<a href="https://colab.research.google.com/github/mhuertascompany/Saas-Fee/blob/main/hands-on/session2/Galaxy_Morphology_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Galaxy Morphology with "classical ML" and deep learning

The goal of this tutorial is to illustrate a very basic supervised binary classification with different ML approaches. The goal is to setup a ML algorithm to determine the visual morphological type of nearby galaxies from the Sloan Digital Sky Survey. The first deep learning papers in Astronomy addressed this problem at low and high redshift (Dielemann+15, Huertas-Company+15).

![](https://drive.google.com/uc?id=1TaiRB1wxui4AKnhuF4iH4LJkmrlb-D6d)

The notebook illustrates first how to train a Random Forest and an Artifical Neural Network using catalog parameters (Stellar Mass and Color). The ML models are taken from scikitlearn.

Then we use a Convolutional Neural Network (deep learning) to learn the features directly from the images with no catalog information. The CNN  model is implemented here with Keras and a TensorFlow backend.  We use as training set, the visually classified sample of ~14,000 galaxies by Nair&Abraham. For illustration purposes, we use jpeg RGB images as input. However the same methodology can be applied to fits.

---



#### Before we start, make sure to open this Colab notebook in "PlayGround Mode" (top left) and to change the Runtime type to GPU by navigating to the toolbar and clicking Runtime -> Change runtime type and then changing Hardware accelerator to GPU

---

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import os
from sklearn import preprocessing

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import  BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPool2D
from sklearn.model_selection import train_test_split


from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score,auc

%pylab inline

## Mount Drive

Before mounting the drive click on [this folder](https://drive.google.com/drive/folders/1PcftgBzBySo1Ync-Wdsp9arTCJ_MfEPE?usp=sharing) and add it to your google drive by following these steps:

*   Go to your drive 
*   Find shared folder ("Shared with me" link)
*   Right click it
*   Click Add to My Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---
#### The notenook is setup to illustrate 2 different classifications:


#### 1.   Early vs. Late: This is an easy example in which we only try to separate between early-type and late-type galaxies.

#### 2.   E vs. S0: The second example is more challenging. We try to separate ellipticals from S0s.

#### By default case 1 is turn on. In order to switch to case 2 set the variable CLASS_EARLY_LATE to False.

---





In [ ]:
CLASS_EARLY_LATE=True

## Deep Learning (CNN)
This section illustrates how to use a Convolutional Neural Network to estimate the visual morphology. The input of the network are now the images themselves without parameters. The CNN then automatically extracts the features. This is the main difference between classical and deep learning.

### Data Download and Preparation

In [ ]:
if CLASS_EARLY_LATE:
  #Load data
  pathinData="/content/drive/My Drive/EDE2019/morphology"

  #images
  X = np.load(pathinData+'/image_vector_E_S_10000.npy')
  #morphological class
  Y = np.load(pathinData+'/target_vector_E_S_10000.npy') 

else:
  #Load data
  pathinData="/content/drive/My Drive/EDE2019/morphology"

  #images
  X = np.load(pathinData+'/image_vector_E_S0_5250.npy')
  #morphological class
  Y = np.load(pathinData+'/target_vector_E_S0_5250.npy')


In [ ]:
# Spliting in Training and Test datasets, 4/5 of galaxies for training, 1/5 for testing
x_train = X[0:len(X)//5*4,:,:,:]   
x_test = X[len(X)//5*4:,:,:,:]
t_train = Y[0:len(Y)//5*4]
t_test = Y[len(Y)//5*4:]
print ('Y_train.shape= ', t_train.shape)          
print(np.max(t_train),np.min(t_train))

### Visualization of images

In [ ]:
randomized_inds_train = np.random.permutation(len(x_train))

fig = plt.figure()
for i,j in zip(randomized_inds_train[0:4],range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(x_train[i,:,:].astype(int))
  plt.title('$Morph$='+str(t_train[i]))
  fig.tight_layout() 
  fig.colorbar(im)


### Pre-processing of images (Normlaziation)

In [ ]:
mu = np.amax(x_train,axis=(1,2))
for i in range(0,mu.shape[0]):
    x_train[i,:,:,0] = x_train[i,:,:,0]/mu[i,0]
    x_train[i,:,:,1] = x_train[i,:,:,1]/mu[i,1]
    x_train[i,:,:,2] = x_train[i,:,:,2]/mu[i,2]
    
mu_test = np.amax(x_test,axis=(1,2))
for i in range(0,mu_test.shape[0]):
    x_test[i,:,:,0] = x_test[i,:,:,0]/mu_test[i,0]
    x_test[i,:,:,1] = x_test[i,:,:,1]/mu_test[i,1]
    x_test[i,:,:,2] = x_test[i,:,:,2]/mu_test[i,2]    


### Model definition
The following cell defines the model architecture. There is a Convolutional part made of multiple convolutinal layers with differnet depths and filter sizes. MaxPooling are used to reduce the sizes of tensors as we go deeper into the network. Batch Normalization and Dropout are use at the training phase to reduce over fitting.

In [ ]:
## THIS IS THE MAIN CELL TO DEFINE A CONVOLUTIONAL MODEL
## FELL FREE TO EDIT / CHANGE / ADD LAYERS
## AS IT IS NOW IT CONTAINS ONLY ONE COVOLUTIONAL LAYER

def CNN_Nair(img_channels, img_rows, img_cols):
    
    # some hyperparamters you can chage
    dropoutpar=0.5
    depth=16   
    nb_dense = 64
    
    model=Sequential()
    model.add(Conv2D(32, 6,6, activation='relu',padding='same',input_shape=(img_rows, img_cols,img_channels)))
   
    model.add(BatchNormalization())  ## batch normalization layer

    ## ADD LAYERS / CONVOLUTIONS HERE
    
    ## FROM HERE IS THE FULLY CONNECTED PART OF THE MODEL. YOU CAN ALSO CHANGE THIS
    model.add(Flatten())
    model.add(Dense(nb_dense, activation='relu'))
    model.add(Dropout(dropoutpar)) 

    ## OUTPUT LAYER -- DO NOT CHANGE
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))  

    # COMPILES THE MODEL. WE USE CROSS-ENTROPY LOSS. YOU CAN CHANGE THE OPTIMIZER
    # IF NEEDED
    print("Compilation...")
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    print("... done!")
    print("Model Summary")
    print("===================")
    model.summary()
    return model

This cell is to delete prevois runs if needed. Only run if you want to delete the results from previous runs.

In [ ]:
RESET=True
if RESET:
  os.system("rm -r "+pathout)

Run this cell to launch a TensorBoard panel which will allow to follow the progress of the training of the deep neural network. Just run the cell and you should see an orange panel appearing. If not try again a second time.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir morphology/models/cnn1

Main code for training the CNN model defined previously. All outputs are stored in pathout. Change if you want to keep different models. BY default the dataset is augmented to reduce overfitting. You can turn it off be setting data_augmentation to False.

In [ ]:
# train params - hardocded for simplicity - you can change them
pathout='morphology/models/model1' #output folder to store the results
model_name = '/cnn1'  #name of the final model which is saved in pathout

LOAD_MODEL=False

batch_size = 32 
nb_epoch = 20  #number of iterations for training - Can be changed
data_augmentation = True # if set to True the data will be augmented at every iteration


    
# input image dimensions
img_rows, img_cols = x_train.shape[1:3]
img_channels = 3

#Avoid more iterations once convergence
patience_par=10
earlystopping = EarlyStopping(monitor='val_loss',patience = patience_par,verbose=0,mode='auto' )
modelcheckpoint = ModelCheckpoint(pathout+model_name+"_best.hd5",monitor='val_loss',verbose=0,save_best_only=True)
tensorboard = TensorBoard(log_dir=pathout)

#build model
cnn=CNN_Nair(img_channels, img_rows, img_cols)
if LOAD_MODEL:
  cnn.load_weights(pathout+model_name+'.hd5')


if not data_augmentation:
  print('Not using data augmentation.')
  history = cnn.fit(x_train, t_train,
                            batch_size=batch_size,
                            nb_epoch=nb_epoch,
                            validation_data=(x_test, t_test),
                            shuffle=True,
                            verbose=verbose, callbacks=[earlystopping, modelcheckpoint,tensorboard])
else:
  print('Using real-time data augmentation.')

  # this will do preprocessing and realtime data augmentation. FEEL FREE TO
  # CHANGE THE PARAMETERS AND SEE THE EFFECTS
  datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False, 
            featurewise_std_normalization=False, 
            samplewise_std_normalization=False,
            zca_whitening=False, 
            rotation_range=25,
            width_shift_range=0.1,  
            height_shift_range=0.1, 
            horizontal_flip=True,
            vertical_flip=True,
            zoom_range=[0.75,1.3])  

        
  datagen.fit(x_train)
        
  history = cnn.fit_generator(
                    datagen.flow(x_train, t_train, batch_size=batch_size),
                    steps_per_epoch=batch_size,
                    epochs=nb_epoch,
                    validation_data=(x_test, t_test),
                    callbacks=[ earlystopping, modelcheckpoint,tensorboard]
                )



print("Saving model...")
cnn.save_weights(pathout+model_name+".hd5",overwrite=True)
    
        


## Predictions and comparisons of different approaches
The following cells use the trained models (RF,ANN and CNN) to predict the morphological class of the test dataset and compare the performance of the different algorithms. 

In [ ]:
print("Predicting...")
print("====================")
LOAD_MODEL=False
if LOAD_MODEL:
  cnn = tf.keras.models.load_model(pathout+model_name+".hd5")

Y_pred_DL = cnn.predict(x_test)


We now compute the global accuracy as well as ROC and P-R curves. If you are not familiar with these curves please see the lecture slides or click [here](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) 

In [ ]:
#global accuracy

## TRY DIFFERENT MODELS - SEE THE EFFECTS ON THE FINAL ROC CURVES

## ASSESMENT:: COMPARE IN A UNIQUE NOTEBOOK THE 3 METHODS WE HAVE SEEN
## COMMENT 


Y_pred_DL_class=Y_pred_DL*0
Y_pred_DL_class[Y_pred_DL>0.5]=1

print("Global Accuracy CNN:", accuracy_score(t_test, Y_pred_DL_class))




# ROC curve (False positive rate vs. True positive rate)
fpr_DL, tpr_DL, thresholds_DL = roc_curve(t_test, Y_pred_DL)

print("AUC CNN:", auc(fpr_DL, tpr_DL))

#plot ROC
fig = plt.figure() 
title('ROC curve',fontsize=18)
xlabel("FPR", fontsize=20)
ylabel("TPR", fontsize=20)
xlim(0,1)
ylim(0,1)
plot(fpr_DL,tpr_DL,linewidth=3,color='red',label='CNN')

legend(fontsize=14)


# Precision Recall curve (False positive rate vs. True positive rate)
precision_DL, recall_DL, thresholds_DL = precision_recall_curve(t_test, Y_pred_DL)

#plot PR curve
fig = plt.figure() 
title('P-R curve',fontsize=18)
xlabel("Precision", fontsize=20)
ylabel("Recall", fontsize=20)
xlim(0,1)
ylim(0,1)
plot(precision_DL,recall_DL,linewidth=3,color='red',label='CNN')
legend(fontsize=14)

The follwing cells visualize some random examples of bad classifications in order to explore what the network has understood. If you run multiple times the examples will change.

### Bad classifications of CNNs

In [ ]:
# objects classifed as early-types by the CNN but visually classifed as late-types
bad = np.where((Y_pred_DL[:,0]<0.5)&(t_test==1))
randomized_inds_train = np.random.permutation(bad)

fig = plt.figure()
fig.suptitle("Galaxies visually classifed as Class1 but classified as Class0",fontsize=10)
for i,j in zip(randomized_inds_train[0][0:4],range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(x_test[i,:,:])
  plt.title('$Morph$='+str(t_test[i]))
  fig.tight_layout() 
  fig.colorbar(im)



# objects classifed as late-types by the CNN but visually classifed as early-types
bad2 = np.where((Y_pred_DL[:,0]>0.5)&(t_test==0))
randomized_inds_train = np.random.permutation(bad2)

fig = plt.figure()
fig.suptitle("Galaxies visually classifed as Class0 but classified as Class1",fontsize=10)
for i,j in zip(randomized_inds_train[0][0:4],range(4)):
  ax = fig.add_subplot(2, 2, j+1)
  im = ax.imshow(x_test[i,:,:])
  plt.title('$Morph$='+str(t_test[i]))
  fig.tight_layout() 
  fig.colorbar(im)
